In [21]:
## 

import pandas as pd
import datetime

import paramiko
import io
import os 
import numpy as np

from snowflake.connector.pandas_tools import pd_writer
from snowflake.connector.pandas_tools import write_pandas
from snowflake.sqlalchemy import URL
from sqlalchemy import create_engine
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError

In [22]:
engine = create_engine(URL(
        account = 'QCOHCUV-MM41421',
        user = 'SHATTHIK',
        password = 'Shattiker1',
        database = 'PORT',
        schema = 'CASH',
        warehouse = 'COMPUTE_WH'
    ))

In [23]:
CASH = pd.read_csv("cashbal.csv")

In [24]:
SEC = pd.read_csv("secpos.csv")

In [25]:
CASH

,As of date,Account name,Account ID,Nickname,Opening Balance,Current balance,Currency
0,2022-11-11,NORSELAB FUND I,000027503EUR,027503EU,0.00,0.00,EUR
1,2022-11-11,NORSELAB FUND I,000027503GBP,027503GB,0.00,0.00,GBP
2,2022-11-11,NORSELAB FUND I,000027503NOK,027503NO,32365592.89,25795592.89,NOK
3,2022-11-11,NORSELAB FUND I,000027503USD,027503US,-0.07,-0.07,USD
4,2022-11-11,NORSELAB FUND I COLLECTION,000027505EUR,027505EU,0.00,0.00,EUR
5,2022-11-11,NORSELAB FUND I COLLECTION,000027505NOK,027505NO,0.00,0.00,NOK
6,2022-11-11,NORSELAB FUND I COLLECTION,000027505USD,027505US,0.00,0.00,USD
7,2022-11-11,NORSELAB FUND I DISTRIBUTION,000027506EUR,027506EU,0.00,0.00,EUR
8,2022-11-11,NORSELAB FUND I DISTRIBUTION,000027506NOK,027506NO,0.00,0.00,NOK
9,2022-11-11,NORSELAB FUND I DISTRIBUTION,000027506USD,027506US,0.00,0.00,USD


In [26]:
#SEC

In [27]:
CASH2 = CASH[["As of date","Account name","Opening Balance","Current balance","Currency"]].copy()


CASH2["Security name"] = CASH["Currency"]
CASH2["Security ID"] = CASH["Currency"]

CASH2["Total units"] = CASH2["Market value"] = CASH["Current balance"]
CASH2["Unit price"] = 1
CASH2["Unit price currency"] = CASH["Currency"]
CASH2["Asset category"] = "FX"


cash_ledge = CASH2[["As of date","Account name","Security name","Security ID","Total units","Unit price","Market value","Unit price currency","Asset category"]]
sec_ledge = SEC[["As of date","Account name","Security name","Security ID","Total units","Unit price","Market value","Unit price currency","Asset category"]]


PORT = pd.concat([sec_ledge,cash_ledge])
PORT["As of date"] = pd.to_datetime(PORT["As of date"]).dt.date

In [28]:
### lag moc
MPORT = pd.read_excel(r"G:\Shared drives\Norselab - Investment Team\Norselab AS\13 Projects\High Yield\ModelPort\modport_0211.2022.xlsx")
MPORT = MPORT[MPORT.Weight>0].copy()

In [29]:
total_cash = 500_000_000
MPORT["As of date"] = datetime.date.today()
MPORT["Account name"] = "NORSELAB MEANINGFUL IMPACT HIGH YIELD"
MPORT["Security name"] = MPORT["Bond"]
MPORT["Security ID"] = MPORT["ISIN"]
MPORT["Market value"] = total_cash*MPORT["Weight"]/100
MPORT["Unit price"] = 100
MPORT["Total units"] = MPORT["Market value"]/MPORT["Unit price"]
MPORT["Unit price currency"] = MPORT["CUR"]
MPORT["Asset category"] = "FI"




In [30]:
PORT_pos = MPORT[["As of date","Account name","Security name","Security ID","Total units","Unit price","Market value","Unit price currency","Asset category"]]

In [31]:
def gen_cash(d,c):

    cp = {'As of date': d,
    'Account name': 'NORSELAB MEANINGFUL IMPACT HIGH YIELD',
    'Security name': c,
    'Security ID': c,
    'Total units': 0.0,
    'Unit price': 1,
    'Market value': 0.0,
    'Unit price currency': c,
    'Asset category': 'FX'}

    return cp

In [32]:
all_cp = []
for i in ["EUR","NOK","SEK","USD","DKK"]:
    all_cp.append(gen_cash(datetime.date.today(),i))

PORT_CASH = pd.DataFrame(all_cp)


In [33]:
PORT_CASH.loc[PORT_CASH["Security name"]=="NOK","Market value"] = 100_000
PORT_CASH["Total units"] = PORT_CASH["Market value"]

In [34]:
PORT_FIN = pd.concat([PORT_pos,PORT_CASH])

In [35]:
PORT_FIN.columns = [i.upper().replace(" ","_") for i in PORT_FIN.columns]

In [36]:
PORT_FIN

,AS_OF_DATE,ACCOUNT_NAME,SECURITY_NAME,SECURITY_ID,TOTAL_UNITS,UNIT_PRICE,MARKET_VALUE,UNIT_PRICE_CURRENCY,ASSET_CATEGORY
0,2022-11-22,NORSELAB MEANINGFUL IMPACT HIGH YIELD,FERTSA Float 12/22/25,NO0010912801,200000.0,100,20000000.0,EUR,FI
1,2022-11-22,NORSELAB MEANINGFUL IMPACT HIGH YIELD,STORSK Float 12/01/25,SE0017084650,200000.0,100,20000000.0,SEK,FI
2,2022-11-22,NORSELAB MEANINGFUL IMPACT HIGH YIELD,BULKID Float 07/14/23,NO0010886922,150000.0,100,15000000.0,NOK,FI
3,2022-11-22,NORSELAB MEANINGFUL IMPACT HIGH YIELD,SUPOFC Float 11/05/25,NO0010900129,150000.0,100,15000000.0,NOK,FI
4,2022-11-22,NORSELAB MEANINGFUL IMPACT HIGH YIELD,LINKNO 3 ⅜ 12/15/25,NO0010911506,150000.0,100,15000000.0,EUR,FI
5,2022-11-22,NORSELAB MEANINGFUL IMPACT HIGH YIELD,ENDURN Float 03/03/25,NO0010935430,150000.0,100,15000000.0,NOK,FI
6,2022-11-22,NORSELAB MEANINGFUL IMPACT HIGH YIELD,FORCEB Float 07/05/26,SE0016275820,150000.0,100,15000000.0,EUR,FI
7,2022-11-22,NORSELAB MEANINGFUL IMPACT HIGH YIELD,HUMBLE Float 07/21/25,SE0016273742,150000.0,100,15000000.0,SEK,FI
8,2022-11-22,NORSELAB MEANINGFUL IMPACT HIGH YIELD,AKERNO Float 08/15/25,NO0010923220,150000.0,100,15000000.0,NOK,FI
9,2022-11-22,NORSELAB MEANINGFUL IMPACT HIGH YIELD,BMKLN Float 09/27/25,NO0012704099,150000.0,100,15000000.0,NOK,FI


In [37]:
#PORT_FIN.loc[PORT_FIN.security_id == "NOK","total_units"] = 4000000

In [38]:
PORT_FIN.loc[PORT_FIN.SECURITY_ID == "NOK","TOTAL_UNITS"] =500_000_000
PORT_FIN.loc[PORT_FIN.SECURITY_ID == "NOK","MARKET_VALUE"] =500_000_000

In [39]:
PORT_FIN[PORT_FIN.SECURITY_ID.isin(["NOK","EUR","SEK","USD","DKK"])]

,AS_OF_DATE,ACCOUNT_NAME,SECURITY_NAME,SECURITY_ID,TOTAL_UNITS,UNIT_PRICE,MARKET_VALUE,UNIT_PRICE_CURRENCY,ASSET_CATEGORY
0,2022-11-22,NORSELAB MEANINGFUL IMPACT HIGH YIELD,EUR,EUR,0.0,1,0.0,EUR,FX
1,2022-11-22,NORSELAB MEANINGFUL IMPACT HIGH YIELD,NOK,NOK,500000000.0,1,500000000.0,NOK,FX
2,2022-11-22,NORSELAB MEANINGFUL IMPACT HIGH YIELD,SEK,SEK,0.0,1,0.0,SEK,FX
3,2022-11-22,NORSELAB MEANINGFUL IMPACT HIGH YIELD,USD,USD,0.0,1,0.0,USD,FX
4,2022-11-22,NORSELAB MEANINGFUL IMPACT HIGH YIELD,DKK,DKK,0.0,1,0.0,DKK,FX


In [40]:
PORT_FIN[PORT_FIN.SECURITY_ID.isin(["NOK","EUR","SEK","USD","DKK"])].to_sql("portfolio",con = engine,if_exists="append",index=False,method=pd_writer)